In [1]:
import xml.dom.minidom
import xml.sax
'''
使用minidom解析器打开 XML 文档，当然使用sax也行。或者也可以尝试自己写解析算法，比如正则表达式。
不同的方法实现得好都有机会被作为作业样本并获得加分
'''
def get_poems():
    DOMTree = xml.dom.minidom.parse("tang300.xml")
    collection = DOMTree.documentElement

    poems = collection.getElementsByTagName("作业用唐诗")
    poemslen = len(poems)
    saved_poems = {}
    for i, poem in enumerate(poems):#枚举每一行
        line = poem.getElementsByTagName("contance")[0].childNodes[0].data
        if  line.startswith('$$'):#如果作者对应上了
            author = line[2:]
            poem_tuple = []
            
            Poem_id = poem.getElementsByTagName("Poem_id")[0].childNodes[0].data
            j = i - 1   #作者上一行是标题
            while j < poemslen and poems[j].getElementsByTagName("Poem_id")[0].childNodes[0].data == Poem_id:#把整首诗存在poem_tuple中
                start = 0
                if j <= i:
                    start = 2    #去除标题和作者名字的前缀
                poem_tuple.append(poems[j].getElementsByTagName("contance")[0].childNodes[0].data[start:])
                j += 1
            if author not in saved_poems.keys():
                saved_poems[author] = []
            saved_poems[author].append(tuple(poem_tuple))
                
    return saved_poems

### 发布者-订阅者模式

实现 设计模式中的观察者模式，本部分将以第一部分中提取的诗歌信息作为主题。

观察者模式也被称为发布-订阅（Publish/Subscribe）模式。当这个主题对象状态变化时，会通知所有观察者对象并作出相应处理逻辑。

在本问题中，可以形象地理解成，Publisher为诗歌发布中心，Reader为订阅相应诗人的诗歌的人。需要实现订阅，退订，Publisher可以发布新增、删除诗歌的消息以及当前各位诗人的诗歌的总数。当Publisher发布消息时，它需要把需要推送给所有订阅的Reader。


**Publisher（发布者）**

    - 保存订阅的读者
    - 记录四位诗人各自诗歌的总数
    - 发布四位诗人新诗的消息，每次只需要发布一首
    - 发布新消息时，发布给所有已订阅的读者   
   
**Reader（订阅者）**

    - 订阅/取消订阅 发布者
    - 保存发布者发来的诗歌
    - 保存所有消息，这意味着这类读者需要将四位诗人的诗歌分类。
 


#### 参考资料

[1. Observer pattern Wiki](https://en.wikipedia.org/wiki/Observer_pattern)   
[2. 设计模式之观察者模式（c++）](https://www.cnblogs.com/carsonzhu/p/5770253.html)

In [2]:
import threading

In [3]:
class BasePublisher(object):
    def __init__(self, name, author_list):
        self.authors = author_list
        self.readers = {x:[] for x in author_list}
        self.name=name
        self.totalNumber={x:0 for x in author_list}
        self.poems = []
    def subscribeReader(self, author, reader):
        self.readers[author].append(reader)
        return self

    def unsubscribeReader(self, author, reader):
        self.readers[author].remove(reader)
        return self

    def notifyReader(self,author, poem):
        pass
    
    def __str__(self):
        readers = f'Subscribed Reader: {set([reader.name for author in self.authors for reader in self.readers[author]])}\n'
        poems = f'发出的诗歌:{[(key,value) for key,value in self.totalNumber.items()]}\n'
        return readers + poems

class Publisher(BasePublisher):
    def __init__(self, name, author_list):
        BasePublisher.__init__(self, name, author_list)
        
    def notifyReader(self, author, poem):
        self.totalNumber[author] += 1
        news = '{}的诗{}'.format(author,poem)
        self.poems.append(news)
        for reader in self.readers[author]:
            reader.receivePoem(self, poem, author)

In [4]:
class BaseReader(object):

    def __init__(self):
        # BaseReader 的初始化方法
        self.name=""
        self.publishers=[]
        self.authors = []
        
    def subscribeToPublisher(self, author, publisher):
        # Reader向Publisher订阅
        publisher.subscribeReader(author, self)
        self.publishers.append(publisher)
        self.authors.append(author)
        
    def unsubscribeToPublisher(self, author, publisher):
        # Reader向Publisher取消订阅
        publisher.unsubscribeReader(author, self)
        self.authors.remove(author)
        
    def __str__(self):
        pubs = f'Subscribed Publisher: {[publisher.name for publisher in self.publishers]}\n'
        return pubs
    
    def receivePoem(self, publisher, poem , author):
        pass
    
    def printStatistics(self):
        # 打印消息
        pass

In [5]:
class Reader(BaseReader):

    def __init__(self, _name):
        BaseReader.__init__(self)
        self.name=_name
        self.poems =  {}
        
    def receivePoem(self, publisher, poem, author):
        if author not in self.poems.keys():
            self.poems[author] = []
        if poem not in self.poems[author]:
            self.poems[author].append(poem)
        
    def printStatistics(self):
        print("{}存有".format(self.name))
        for key in self.poems.keys():
            if (len(self.poems[key])>0):
                print(f'诗人{key}的诗有:')
                for poem in self.poems[key]:
                    print(poem)

In [6]:
if __name__ == "__main__":
    poem_dist = get_poems()
    
    publisher = Publisher("Publisher", list(poem_dist.keys()))
    Alice=Reader('Alice')
    Bob=Reader('Bob')
    
    Alice.subscribeToPublisher("李白", publisher)
    Bob.subscribeToPublisher("杜甫",publisher)
    
    publisher.notifyReader("李白",poem_dist["李白"][0])
    publisher.notifyReader("李白",poem_dist["李白"][1])
    publisher.notifyReader("李白",poem_dist["李白"][2])
    publisher.notifyReader("李白",poem_dist["李白"][3])
    publisher.notifyReader("李白",poem_dist["李白"][4])
    
    Alice.printStatistics()
    Bob.printStatistics()
    
    publisher.notifyReader("杜甫",poem_dist["杜甫"][1])
   
    Alice.printStatistics()
    Bob.printStatistics()
    
    Bob.unsubscribeToPublisher("杜甫", publisher)
    
    publisher.notifyReader("白居易",poem_dist["白居易"][2])
    print(publisher)

Alice存有
诗人李白的诗有:
('登峴山亭寄晉陵張少府', '李白', '峴首風湍急', '雲帆若鳥飛', '憑軒試一問', '張翰欲來歸')
('送朱大入秦', '李白', '遊人武陵去', '寶劍直千金', '分手脫相贈', '平生一片心')
('結襪子', '李白', '燕南壯士吳門豪', '築中置鉛魚隱刀', '感君恩重許君命', '太山一擲輕鴻毛')
('玉階怨', '李白', '玉階生白露', '夜久侵羅襪', '卻下水晶簾', '玲瓏望秋月')
('洛陽陌', '李白', '白玉誰家郎', '迴車渡天津', '看花東陌上', '驚動洛陽人')
Bob存有
Alice存有
诗人李白的诗有:
('登峴山亭寄晉陵張少府', '李白', '峴首風湍急', '雲帆若鳥飛', '憑軒試一問', '張翰欲來歸')
('送朱大入秦', '李白', '遊人武陵去', '寶劍直千金', '分手脫相贈', '平生一片心')
('結襪子', '李白', '燕南壯士吳門豪', '築中置鉛魚隱刀', '感君恩重許君命', '太山一擲輕鴻毛')
('玉階怨', '李白', '玉階生白露', '夜久侵羅襪', '卻下水晶簾', '玲瓏望秋月')
('洛陽陌', '李白', '白玉誰家郎', '迴車渡天津', '看花東陌上', '驚動洛陽人')
Bob存有
诗人杜甫的诗有:
('因崔五侍御寄高彭州  適', '杜甫', '百年已過半', '秋至轉饑寒', '為問彭州牧', '何時救急難')
Subscribed Reader: {'Alice'}
发出的诗歌:[('沈佺期', 0), ('尹懋', 0), ('楊重玄', 0), ('張循之', 0), ('劉幽求', 0), ('章玄同', 0), ('盧僎', 0), ('李崇嗣', 0), ('呂太一', 0), ('張紘', 0), ('武平一', 0), ('趙彥昭', 0), ('蕭至忠', 0), ('趙彥伯', 0), ('鄭愔', 0), ('徐堅', 0), ('李適之', 0), ('張諤', 0), ('劉庭琦', 0), ('鄭繇', 0), ('袁暉', 0), ('賀知章', 0), ('包融', 0), ('蔡希寂', 0), ('張潮', 0), ('張翬', 0), ('周瑀', 0), ('沈如